# Aula 05

## MVC

O MVC é um um padrão de arquitetura de software que divide uma aplicação em três camadas/componentes interconectados: 

- **M**odel (Modelo) 
  - Responsável pela lógica de negócios e dados.
  - Responsável, também, por buscar, processar e retornar dados. Para isso, é esse componente que interage com a fonte de dados, por exemplo, um banco de dados.
- **V**iew (Visão)
  - Lida com a apresentação dos dados ao usuário.
  - Não contém lógica de negócios e não processa dados diretamente.
  - Pode ser um um layout ou template que recebe os dados do Controller.
- **C**ontroller (Controlador)
  - É o intermediário entre o Model e a View.
  - Recebe requisições do usuário e as direciona para o Model.
  - Recebe os dados do Model e seleciona a View adequada para exibir a resposta ao usuário.
  
![Fonte: Python Tutorial (https://www.pythontutorial.net/wp-content/uploads/2021/10/tkinter-mvc-design-pattern.png)](tkinter-mvc-design-pattern.png)

Aplicação de exemplo: um `widget` `entry` para o usuário inserir um endereço de e-mail, e um `button` para salvar. Quando o usuário clicar no botão o e-mail passa por uma validação. Caso positivo, uma mensagem de sucesso é mostrada; caso negativo, uma mensagem de erro.

### Model

In [ ]:
import re

class Model:
    def __init__(self, email=None):
        if email is None:
            self.__email = email
        else:
            self.email = email

    @property # decorator
    def email(self):
        return self.__email

    @email.setter
    def email(self, value):
        """
        Valida o email com expressão regular (regex)
        """
        pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        if re.fullmatch(pattern, value):
            self.__email = value
        else:
            raise ValueError(f'Endereço de e-mail inválido: {value}')

    def save(self):
        """
        Salva o e-mail em um arquivo de texto
        """
        with open('emails.txt', 'a') as f:
            f.write(self.email + '\n')

### View

In [ ]:
import tkinter as tk
from tkinter import ttk

class View(ttk.Frame):
    def __init__(self, parent):
        super().__init__(parent)

        # Widgets
        #   Label -> para o entry
        self.label = ttk.Label(self, text='Email:')
        self.label.grid(row=1, column=0)

        #   Entry (email)
        self.email_var = tk.StringVar()
        self.email_entry = ttk.Entry(self, textvariable=self.email_var, width=30)
        self.email_entry.grid(row=1, column=1, sticky=tk.NSEW)

        #   Button
        self.save_button = ttk.Button(self, text='Salvar', command=self.save_button_clicked)
        self.save_button.grid(row=1, column=3, padx=10)

        #   Label -> para a messagem de confirmação/erro
        self.message_label = ttk.Label(self, text='', foreground='red')
        self.message_label.grid(row=2, column=1, sticky=tk.W)

        # set the controller
        self.controller = None

    def set_controller(self, controller):
        """
        Define o controller (objeto da classe Controller)
        """
        self.controller = controller

    def save_button_clicked(self):
        """
        Manipula o clique no botão
        """
        if self.controller:
            # Save é um método definido na classe Controller
            self.controller.save(self.email_var.get())

    def show_error(self, message):
        """
        Exibe uma mensagem de erro
        """
        self.message_label['text'] = message
        self.message_label['foreground'] = 'red'
        self.message_label.after(3000, self.hide_message)
        self.email_entry['foreground'] = 'red'

    def show_success(self, message):
        """
        Exibe mensagem de sucesso
        """
        self.message_label['text'] = message
        self.message_label['foreground'] = 'green'
        self.message_label.after(3000, self.hide_message)

        # Reinica o formulário
        self.email_entry['foreground'] = 'black'
        self.email_var.set('')

    def hide_message(self):
        """
        Apaga a mensagem
        """
        self.message_label['text'] = ''

### Controller

In [ ]:
class Controller:
    def __init__(self, model, view):
        self.model = model # objeto da classe Model
        self.view = view   # objeto da classe View

    def save(self, email):
        """
        Salva o e-mail
        """
        try:
            # Salvando o e-mail com o método de Model
            self.model.email = email
            self.model.save()

            # Exibindo uma mesagem de sucesso
            self.view.show_success(f'O e-mail {email} foi salvo!')

        except ValueError as error:
            # Exibindo a mensagem de erro
            self.view.show_error(error)

### Unindo o MVC

In [ ]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title('Tkinter MVC')

        # Instanciando Model
        model = Model()

        # Instanciando View e colocando o objeto na janela root
        view = View(self)
        view.grid(row=0, column=0, padx=10, pady=10)

        # Instanciando Controller
        controller = Controller(model, view)

        # Definindo o objeto controller para o objeto view
        view.set_controller(controller)


if __name__ == '__main__':
    app = App()
    app.mainloop()

## Exercícios

### Fáceis

1. **Hello MVC básico**
   Crie três módulos ou classes: `Model`, `View`, `Controller`. A `View` contém uma janela com um botão “Clique”. Quando clicar, o `Controller` chama o `Model` para obter uma mensagem (“Olá do modelo”) e a `View` a exibe num `Label`.

2. **Contador MVC**
   O `Model` mantém um valor inteiro (inicial 0). A `View` exibe o valor e tem botões “+1” e “–1”. O `Controller` manipula esses eventos alterando o modelo e atualizando a view.

3. **Modelo de texto simples**
   `Model` armazena um texto (string). A `View` mostra um `Entry` e botão “Salvar”. Ao salvar, o `Controller` pede ao `Model` armazenar o texto e mostra confirmação em uma label.

4. **Entrada e exibição MVC**
   `View` com `Entry` e botão “Mostrar”. Ao clicar, `Controller` obtém string do `Entry` e manda `View` exibir em `Label`.

5. **Aplicando cor MVC**
   `View` mostra três botões (“Vermelho”, “Verde”, “Azul”). Ao clicar, `Controller` informa ao `Model` qual cor foi escolhida; o `View` então muda seu fundo para essa cor.

6. **Checkbox MVC**
   `View` apresenta um `Checkbutton` e `Label`. O `Model` mantém valor booleano. Ao mudar o checkbox (`Controller` captura evento), atualize `Model` e faça `View` exibir “Ativado” ou “Desativado”.

7. **Radio MVC**
   Três `Radiobuttons` para escolher “Opção A”, “B”, “C”. O `Model` guarda a escolha. Quando o usuário mudar, o `Controller` informa o `Model` e a `View` mostra a opção atual num label.

8. **Contador automático MVC**
   Use `after()` para incrementar automaticamente o contador no `Model` a cada segundo, e o `Controller` manda a `View` atualizar. A `View` também pode parar/iniciar via botão.

9. **Lista de itens MVC**
   `View` com `Entry` + botão “Adicionar” + `Listbox`. O `Model` mantém uma lista de strings. Ao clicar “Adicionar”, o `Controller` adiciona ao modelo e informa a view para mostrar no `Listbox`.

10. **Remover item MVC**
    Estenda o exercício anterior com um botão “Remover” que exclui o item selecionado no `Listbox`. `Controller` cuida dessa lógica e atualiza modelo e view.

11. **Sincronização de modelo e entrada**
    `Model` tem um campo que é atualizado de fora (simulação). A `View` exibe o valor periodicamente. Use `after()` do `Controller` para sincronizar view ↔ model.

12. **MVC com diálogo de confirmação**
    `View` tem botão “Sair”. Ao clicar, o `Controller` pede confirmação via messagebox; se confirmar, fecha aplicação.

In [ ]:
# ...


### Médios

13. **MVC com validação**
    `View` com `Entry` para número. Quando clicar “Calcular”, o `Controller` verifica se é número válido, informa `Model` ou exibe erro, e `View` exibe resultado ou erro.

14. **Login MVC com credenciais fixas**
    `View` pede usuário e senha. O `Controller` checa (com valores fixos) e, se correto, manda a `View` exibir “Login OK” ou “Falha”.

15. **MVC com múltiplas views (telas)**
    Duas views: tela de login e tela principal. O `Controller` gerencia qual tela mostrar com base no estado do `Model`.

16. **Persistência simples MVC**
    `Model` deve gravar e ler dados de um arquivo JSON ou texto (lista de itens). A `View` permite adicionar/remover itens e salvar/abrir (via menu ou botões). `Controller` gerencia persistência.

17. **Busca/autocompletar MVC**
    `Model` mantém lista de palavras. `View` tem um `Entry` e `Listbox` de sugestões. Ao digitar, o `Controller` pede ao `Model` para filtrar sugestões e exibe-as na view.

18. **MVC com threads**
    `Model` executa uma tarefa lenta (por exemplo, simulação de cálculo intensivo). O `Controller` inicia uma thread para essa tarefa e, quando termina, atualiza `Model` e manda `View` exibir resultado (via `after()`).

19. **Timer com MVC e controle**
    `Model` guarda tempo decorrido; o `Controller` tem métodos start/pause/reset; a `View` mostra o tempo e oferece botões para controle.

20. **Dashboard MVC com gráficos simples**
    `Model` possui dados estatísticos (ex: vendas fictícias por dia). `View` exibe texto ou gráfico (canvas simples). `Controller` permite recarregar dados ou filtrar por período.

In [ ]:
# ...

### Difíceis

21. **Aplicação MVC com múltiplos modelos interdependentes**
    Ex: um modelo de “Usuário” e outro de “Perfil”. A `View` permite gerenciar usuários (criar, editar) e perfis associados. O `Controller` coordena as ações entre modelos e views.

22. **Editor MVC com undo/redo**
    `View` com `Text`, botão “Desfazer”, “Refazer”; `Model` guarda histórico de edições; `Controller` aplica comandos de edição e gerencia o histórico.

23. **Sistema de chat MVC com servidor simulado**
    `Model` simula mensagens enviadas/recebidas; `Controller` em thread comunica com o modelo, recebendo novas mensagens e enviando mensagens do usuário; `View` exibe bate-papo, entrada de texto e botão “Enviar”.

24. **Aplicativo de planilha simples MVC**
    `Model` contém dados tabulares (matriz). `View` exibe células (grid de Entry ou labels). `Controller` permite inserir fórmulas simples ou valores, atualizar células dependentes, salvar/abrir, adicionar/remover linhas/colunas.

In [ ]:
# ...